In [1]:
import pygame
import math
import numpy as np
import sys

pygame.init()
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
clock = pygame.time.Clock()

# obj 파일 불러오기
def load_obj(filename):
    vertices = []
    edges = set()
    faces = []
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('v '):
                parts = line.split()
                x, y, z = map(float, parts[1:4])
                vertices.append(np.array([x, y, z]))
            elif line.startswith('f '):
                parts = line.split()
                indices = [int(part.split('/')[0]) - 1 for part in parts[1:]]
                faces.append(indices)
                for i in range(len(indices)):
                    edge = tuple(sorted((indices[i], indices[(i + 1) % len(indices)])))
                    edges.add(edge)
    return np.array(vertices), list(edges)

# 카메라 설정
camera_pos = np.array([0.0, 0.0, -5.0])
camera_target = np.array([0.0, 0.0, 0.0])
camera_up = np.array([0.0, 1.0, 0.0])

# look-at 뷰 행렬 계산 함수 (뷰 변환 행렬)
def look_at(camera_pos, camera_target, camera_up):
    forward = camera_target - camera_pos
    forward /= np.linalg.norm(forward)

    right = np.cross(camera_up, forward)
    right /= np.linalg.norm(right)

    up = np.cross(forward, right)

    rotation = np.array([right, up, forward])
    translation = -rotation @ camera_pos
    view_matrix = np.eye(4)
    view_matrix[:3, :3] = rotation
    view_matrix[:3, 3] = translation
    return view_matrix

# 3D → 2D 투영 함수
def project(point3d):
    z = point3d[2] + 5  # z는 원근 효과 조절
    if z == 0:
        z = 0.001
    factor = 500 / z
    x = point3d[0] * factor + width // 2
    y = -point3d[1] * factor + height // 2
    return int(x), int(y)

# 회전 행렬
def x_rotate(angle):
    cos_theta = math.cos(angle)
    sin_theta = math.sin(angle)
    return np.array([
        [1, 0, 0],
        [0, cos_theta, -sin_theta],
        [0, sin_theta, cos_theta]
    ])

def y_rotate(angle):
    cos_theta = math.cos(angle)
    sin_theta = math.sin(angle)
    return np.array([
        [cos_theta, 0, sin_theta],
        [0, 1, 0],
        [-sin_theta, 0, cos_theta]
    ])

# obj 파일 경로
obj_path = r"C:\Users\USER\Desktop\학교\2학년\응프\makes_game\model\tinker.obj"
vertices, edges = load_obj(obj_path)

angle_x = 0.0
angle_y = 0.0

# 마우스 커서 숨기고 중앙에 위치시키기, 마우스 고정 상태 변수
grabbed = True
pygame.mouse.set_visible(False)
pygame.event.set_grab(grabbed)
pygame.mouse.set_pos((width // 2, height // 2))

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                # ESC 키로 마우스 고정 토글 및 프로그램 종료
                if grabbed:
                    grabbed = False
                    pygame.event.set_grab(False)
                    pygame.mouse.set_visible(True)
                else:
                    pygame.quit()
                    sys.exit()

    if grabbed:
        mx, my = pygame.mouse.get_pos()
        dx = mx - width // 2
        dy = my - height // 2

        sensitivity = 0.002
        angle_y += dx * sensitivity
        angle_x += dy * sensitivity

        pygame.mouse.set_pos((width // 2, height // 2))
    else:
        # 마우스 고정 해제 상태에서는 회전하지 않음
        dx = dy = 0

    rotation = x_rotate(angle_x) @ y_rotate(angle_y)
    view_matrix = look_at(camera_pos, camera_target, camera_up)

    transformed_vertex = []
    for v in vertices:
        rotated = rotation @ v
        vec4 = np.append(rotated, 1)
        camera_space = view_matrix @ vec4
        camera_space_3d = camera_space[:3]
        projected = project(camera_space_3d)
        transformed_vertex.append(projected)

    screen.fill((0, 0, 0))
    for edge in edges:
        pygame.draw.line(screen, (255, 255, 255),
                         transformed_vertex[edge[0]],
                         transformed_vertex[edge[1]], 2)

    pygame.display.flip()
    clock.tick(60)


pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
